In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
import numpy as np

In [4]:
ratings = tfds.load("movie_lens/100k-ratings", split="train")

Shuffling and writing examples to /Users/avilay/tensorflow_datasets/movie_lens/100k-ratings/0.1.0.incompleteEMAFKZ/movie_lens-train.tfrecord


Dataset movie_lens downloaded and prepared to /Users/avilay/tensorflow_datasets/movie_lens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


In [5]:
movies = tfds.load("movie_lens/100k-movies", split="train")

Shuffling and writing examples to /Users/avilay/tensorflow_datasets/movie_lens/100k-movies/0.1.0.incompleteOKMOX9/movie_lens-train.tfrecord


Dataset movie_lens downloaded and prepared to /Users/avilay/tensorflow_datasets/movie_lens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [6]:
type(ratings)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [8]:
next(iter(ratings))

{'bucketized_user_age': <tf.Tensor: shape=(), dtype=float32, numpy=45.0>,
 'movie_genres': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([7])>,
 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'357'>,
 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b"One Flew Over the Cuckoo's Nest (1975)">,
 'raw_user_age': <tf.Tensor: shape=(), dtype=float32, numpy=46.0>,
 'timestamp': <tf.Tensor: shape=(), dtype=int64, numpy=879024327>,
 'user_gender': <tf.Tensor: shape=(), dtype=bool, numpy=True>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'138'>,
 'user_occupation_label': <tf.Tensor: shape=(), dtype=int64, numpy=4>,
 'user_occupation_text': <tf.Tensor: shape=(), dtype=string, numpy=b'doctor'>,
 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=4.0>,
 'user_zip_code': <tf.Tensor: shape=(), dtype=string, numpy=b'53211'>}

In [9]:
next(iter(movies))

{'movie_genres': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([4])>,
 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'1681'>,
 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'You So Crazy (1994)'>}

In [10]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [11]:
user_ids_vocab = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocab.adapt(ratings.map(lambda x: x["user_id"]))

In [13]:
movie_titles_vocab = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocab.adapt(movies)

In [14]:
class MovieLensModel(tfrs.Model):
    def __init__(self, user_model, movie_model, task):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        self.task = task
        
    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])
        return self.task(user_embeddings, movie_embeddings)

In [15]:
user_model = tf.keras.Sequential([
    user_ids_vocab,
    tf.keras.layers.Embedding(user_ids_vocab.vocab_size(), 64),
])

movie_model = tf.keras.Sequential([
    movie_titles_vocab,
    tf.keras.layers.Embedding(movie_titles_vocab.vocab_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(movies.batch(128).map(movie_model)))

In [16]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [17]:
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 5s 183ms/step - factorized_top_k: 0.0300 - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0051 - factorized_top_k/top_50_categorical_accuracy: 0.0443 - factorized_top_k/top_100_categorical_accuracy: 0.0990 - loss: 33085.4280 - regularization_loss: 0.0000e+00 - total_loss: 33085.4280
Epoch 2/3
25/25 [==============================] - 5s 183ms/step - factorized_top_k: 0.0676 - factorized_top_k/top_1_categorical_accuracy: 1.9000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0052 - factorized_top_k/top_10_categorical_accuracy: 0.0152 - factorized_top_k/top_50_categorical_accuracy: 0.1050 - factorized_top_k/top_100_categorical_accuracy: 0.2125 - loss: 31016.2948 - regularization_loss: 0.0000e+00 - total_loss: 31016.2948
Epoch 3/3
25/25 [==============================] - 5s 182ms/step - factorized_top_k: 0.0887 - factorized

In [18]:
index = tfrs.layers.ann.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

In [20]:
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[:3]}")

Top 3 recommendations for user 42: [[b'Rent-a-Kid (1995)' b'Just Cause (1995)' b'Princess Caraboo (1994)'
  b'Land Before Time III: The Time of the Great Giving (1995) (V)'
  b'Aristocats, The (1970)' b'Ghost and Mrs. Muir, The (1947)'
  b'All Dogs Go to Heaven 2 (1996)' b'Outbreak (1995)' b'Nell (1994)'
  b'Mad Love (1995)']]
